In [62]:
#!/usr/bin/python3

from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
import matplotlib.pyplot as plt
import subprocess as subp
import os
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

def getWavFiles():
    files = subp.check_output(["ls"]).decode("utf8").split("\n")
    files = [f for f in files if f.split(".")[-1] == "wav"]
    return files

# Gets feature vector for a wav song
def getFeatureVector(file):
    [Fs, x] = audioBasicIO.readAudioFile(file);
    x = audioBasicIO.stereo2mono(x)

    # We cut the audio to a 30 second window in the middle of the song
    # If the audio is shorter than 30 second, we discard it
    nSamples = Fs * 30
    if len(x) < nSamples:
        return None
    offset   = (len(x) - nSamples) // 2
    x = x[offset:offset+nSamples]

    print("Sampling rate, number of samples: ", Fs, len(x))

    mterm, sterm, f_names = audioFeatureExtraction.mtFeatureExtraction(x, Fs, 2*Fs, 2*Fs, 0.050*Fs, 0.025*Fs);

    # Should return 68 mid-term features per mid-term window (30 windows)
    return mterm.ravel()

def getDataset():
    dataset = []

    os.chdir("music_negative/")
    files = getWavFiles()
    for f in files:
        features = getFeatureVector(f)
        if features is not None:
            dataset.append([features, 0])
    os.chdir("..")

    os.chdir("music_positive/")
    files = getWavFiles()
    for f in files:
        features = getFeatureVector(f)
        if features is not None:
            dataset.append([features, 1])
    os.chdir("..")

    return np.array(dataset)

def trainModel(trainData, hidden=(10)):
    clf = MLPClassifier(solver="sgd", alpha=1e-4, hidden_layer_sizes=hidden, random_state=1, max_iter=10000)

    X = [ i[0] for i in trainData ]
    Y = [ i[1] for i in trainData ]
    clf.fit(X, Y)

    return clf

In [4]:
dataset = getDataset()

# Shuffle dataset
np.random.shuffle(dataset)

Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, number of samples:  44100 1323000
Sampling rate, numbe

In [66]:
# Separate in train/test, 70% train
# partition = int(0.7*len(dataset))
# train = dataset[:partition]
# test  = dataset[partition:]

nFolds = 5
foldSize = len(dataset)//nFolds

for layers in [(1), (2), (3), (4), (5), (10), (20), (30), (40), (50), (60), (70), (80), (90), (100), (5,2), (5,4), (10,4), (20,4)]:
    success = 0
    attempts = 0
    for i in range(nFolds):
        train_idx = np.array([ True for i in range(len(dataset)) ])
        train_idx[i*foldSize:(i+1)*foldSize] = False

        if i == nFolds-1:
            train_idx[-1] = False

        test_idx = train_idx == False

        train = dataset[train_idx]
        test  = dataset[test_idx]

        # train model
        clf = trainModel(train, layers)

        # Calculate accuracy
        for [x,y] in test:
            y_star = clf.predict([x])
            # print("[{},{}]".format(y,y_star))
            if y_star == y:
                success += 1
            attempts += 1

    print("{}\t{}".format(layers, success / attempts))

1	0.6774193548387096
2	0.8709677419354839
3	0.5161290322580645
4	0.5161290322580645
5	0.8709677419354839
10	0.8709677419354839
20	0.8387096774193549
30	0.8387096774193549
40	0.8709677419354839
50	0.7741935483870968
60	0.7741935483870968
70	0.8709677419354839
80	0.7096774193548387
90	0.7096774193548387
100	0.5483870967741935
(5, 2)	0.7741935483870968
(5, 4)	0.6129032258064516
(10, 4)	0.8064516129032258
(20, 4)	0.41935483870967744


array([], dtype=float64)